# ノイズ検出 + 復元モデルの学習（Google Colab版）

このノートブックは、ノイズ検出 + 復元タスクの学習を行います。

## 📋 セットアップ手順

1. **GPUを有効化**: ランタイム → ランタイムのタイプを変更 → GPU（T4）を選択
2. **Google Driveをマウント**（推奨）:
   - プロジェクトフォルダをGoogle Driveにアップロード
   - セル1でDriveをマウント
3. **データセットの準備**:
   - セル4でデータセットを生成（`baseline_dataset.pickle`）
   - ノイズタイプ: 新しい3種類のノイズ（デフォルト: `power_supply`）
   - 区間数: 30区間
4. 各セルを順番に実行してください

## 📝 タスクについて

**ノイズ検出 + 復元タスク**:
1. **マスク予測**: どの区間にノイズがあるかを予測（30区間のノイズ強度）
2. **復元**: 予測した区間のみ（100ポイント）を復元
3. **損失**: マスク予測が正しい場合のみ復元損失を計算

## 📝 ノイズタイプについて

現在使用しているノイズタイプ（新しい3種類）:
- **`power_supply`**: 電源ノイズ（デフォルト、2kHzにピーク）
- **`interference`**: 干渉ノイズ（3kHzにピーク）
- **`clock_leakage`**: クロックリークノイズ（5kHzにピーク）

**設定方法**:
- `dataset.py`の`NOISE_TYPE`で選択（デフォルト: `power_supply`）
- `USE_RANDOM_NOISE = True`にすると3種類をランダムに使用


In [ ]:
# Google Driveをマウント（推奨）
from google.colab import drive
drive.mount('/content/drive')

# 作業ディレクトリを変更（プロジェクトフォルダに移動）
import os
os.chdir('/content/drive/MyDrive/soturon')  # プロジェクトフォルダのパス

print(f"現在の作業ディレクトリ: {os.getcwd()}")
print(f"ファイル一覧: {os.listdir('.')[:10]}")  # 最初の10ファイルを表示


## 📦 ライブラリのインストールとインポート


In [ ]:
# ライブラリのインストールとインポート
!pip install torch torchvision scikit-learn matplotlib -q

import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import platform
import os
import sys
import time
import warnings

# matplotlibのキャッシュをクリア（フォント警告を防ぐため）
import shutil
try:
    shutil.rmtree('/root/.cache/matplotlib', ignore_errors=True)
except:
    pass

# matplotlibの警告を抑制
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# プロジェクトルートをパスに追加
project_root = os.getcwd()
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, '畳み込み'))
sys.path.insert(0, os.path.join(project_root, 'ノイズの付与(共通)'))

# GPUの確認
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# 日本語フォントの設定（Colab用）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
# データセットの準備
# 畳み込み/dataset.py を実行してデータセットを生成
print("データセットを準備中...")
print("=" * 60)

# プロジェクトルートとnoiseモジュールのパスを追加
project_root = os.getcwd()
noise_folder_path = os.path.join(project_root, 'ノイズの付与(共通)')
noise_link_path = os.path.join(project_root, 'noise')

# プロジェクトルートをsys.pathに追加（dataset.pyの中でproject_rootからnoiseをインポートするため）
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# ノイズの付与(共通)フォルダをnoiseモジュールとして登録
# 注意: "noise"フォルダは、dataset.pyが「from noise.add_noise import add_noise_to_interval」と
# インポートするために必要な仮想的なパッケージ名です。
# 実際のフォルダ名は「ノイズの付与(共通)」ですが、Pythonのインポートでは「noise」として扱います。
print("noiseモジュールをsys.modulesに登録中...")

# 既存のシンボリックリンクを削除（古いファイルを指している可能性があるため）
if os.path.exists(noise_link_path):
    try:
        if os.path.islink(noise_link_path):
            os.unlink(noise_link_path)
            print(f"古いシンボリックリンクを削除: {noise_link_path}")
    except Exception as e:
        print(f"シンボリックリンクの削除に失敗（無視します）: {e}")

# 既存のnoiseモジュールを削除（キャッシュをクリア）
if 'noise' in sys.modules:
    del sys.modules['noise']
if 'noise.add_noise' in sys.modules:
    del sys.modules['noise.add_noise']

# __pycache__をクリア（古いキャッシュを削除）
import shutil
pycache_path = os.path.join(noise_folder_path, '__pycache__')
if os.path.exists(pycache_path):
    try:
        shutil.rmtree(pycache_path)
        print(f"古いキャッシュを削除: {pycache_path}")
    except Exception as e:
        print(f"キャッシュの削除に失敗（無視します）: {e}")

# __init__.pyの内容を確認し、必要に応じて修正
init_path = os.path.join(noise_folder_path, '__init__.py')
print(f"  __init__.pyのパス: {init_path}")
print(f"  ファイルが存在するか: {os.path.exists(init_path)}")

# Google Colab上のファイルが古い可能性があるため、正しい内容を書き込む
correct_init_content = '''"""
測定系由来のノイズを付与するモジュール
"""

from .power_supply_noise import add_power_supply_noise
from .interference_noise import add_interference_noise
from .clock_leakage_noise import add_clock_leakage_noise
from .add_noise import add_noise_to_interval

__all__ = [
    'add_power_supply_noise',
    'add_interference_noise',
    'add_clock_leakage_noise',
    'add_noise_to_interval'
]
'''

# __init__.pyの内容を確認
try:
    with open(init_path, 'r', encoding='utf-8') as f:
        current_content = f.read()
    
    # 古い内容が含まれている場合は書き換える
    if 'frequency_band_noise' in current_content or 'localized_spike_noise' in current_content:
        print("  古い__init__.pyを検出しました。正しい内容に書き換えます...")
        with open(init_path, 'w', encoding='utf-8') as f:
            f.write(correct_init_content)
        print("  ✓ __init__.pyを更新しました")
    else:
        print("  ✓ __init__.pyは既に正しい内容です")
except Exception as e:
    print(f"  ⚠ __init__.pyの確認に失敗しましたが、続行します: {e}")

# ノイズフォルダをsys.pathに追加（相対インポートを解決するため）
if noise_folder_path not in sys.path:
    sys.path.insert(0, noise_folder_path)

import importlib.util

# noiseパッケージの__init__.pyをロード
init_spec = importlib.util.spec_from_file_location("noise", init_path)
init_module = importlib.util.module_from_spec(init_spec)
sys.modules['noise'] = init_module
# __file__と__path__を設定（相対インポートのため）
init_module.__file__ = init_path
init_module.__path__ = [noise_folder_path]
init_spec.loader.exec_module(init_module)

# add_noise.pyをロード
add_noise_path = os.path.join(noise_folder_path, 'add_noise.py')
add_noise_spec = importlib.util.spec_from_file_location("noise.add_noise", add_noise_path)
add_noise_module = importlib.util.module_from_spec(add_noise_spec)
sys.modules['noise.add_noise'] = add_noise_module
add_noise_module.__file__ = add_noise_path
add_noise_spec.loader.exec_module(add_noise_module)

print("✓ noiseモジュールをsys.modulesに登録しました")
print(f"  利用可能な関数: {init_module.__all__}")

# __file__を定義（exec()で実行する場合に必要）
dataset_path = os.path.join(project_root, '畳み込み', 'dataset.py')

# dataset.pyの内容を確認し、必要に応じて修正
print("\ndataset.pyの内容を確認中...")
try:
    with open(dataset_path, 'r', encoding='utf-8') as f:
        dataset_content = f.read()
    
    # 古いコード（label_counts[i]を直接アクセス）が含まれている場合は修正
    if 'label_counts = torch.bincount(train_labels)' in dataset_content and \
       'for i in range(NUM_INTERVALS):' in dataset_content and \
       'label_counts[i].item()' in dataset_content and \
       'label_counts_full' not in dataset_content:
        print("  古いdataset.pyを検出しました。正しい内容に書き換えます...")
        
        # 修正箇所を特定して置換
        old_pattern = '''# ラベルの分布を確認
print("\\nラベルの分布（訓練データ）:")
label_counts = torch.bincount(train_labels)
for i in range(NUM_INTERVALS):
    print(f"  区間 {i+1:2d}: {label_counts[i].item():4d}サンプル")'''
        
        new_pattern = '''# ラベルの分布を確認
print("\\nラベルの分布（訓練データ）:")
label_counts_full = torch.bincount(train_labels)
# NUM_INTERVALSのサイズに合わせて拡張（存在しない区間は0）
label_counts = torch.zeros(NUM_INTERVALS, dtype=torch.long)
for i in range(min(NUM_INTERVALS, len(label_counts_full))):
    label_counts[i] = label_counts_full[i]
for i in range(NUM_INTERVALS):
    print(f"  区間 {i+1:2d}: {label_counts[i].item():4d}サンプル")'''
        
        if old_pattern in dataset_content:
            dataset_content = dataset_content.replace(old_pattern, new_pattern)
            with open(dataset_path, 'w', encoding='utf-8') as f:
                f.write(dataset_content)
            print("  ✓ dataset.pyを更新しました")
        else:
            # パターンが完全に一致しない場合、より柔軟な置換を試みる
            import re
            # label_counts = torch.bincount(train_labels)の直後に修正を挿入
            pattern = r'(label_counts = torch\.bincount\(train_labels\))\s*(for i in range\(NUM_INTERVALS\):)'
            replacement = r'''label_counts_full = torch.bincount(train_labels)
# NUM_INTERVALSのサイズに合わせて拡張（存在しない区間は0）
label_counts = torch.zeros(NUM_INTERVALS, dtype=torch.long)
for i in range(min(NUM_INTERVALS, len(label_counts_full))):
    label_counts[i] = label_counts_full[i]
\2'''
            if re.search(pattern, dataset_content):
                dataset_content = re.sub(pattern, replacement, dataset_content)
                with open(dataset_path, 'w', encoding='utf-8') as f:
                    f.write(dataset_content)
                print("  ✓ dataset.pyを更新しました（正規表現で修正）")
            else:
                print("  ⚠ dataset.pyの修正パターンが見つかりませんでした")
    else:
        print("  ✓ dataset.pyは既に正しい内容です")
except Exception as e:
    print(f"  ⚠ dataset.pyの確認に失敗しましたが、続行します: {e}")

# グローバル名前空間に__file__を追加して実行
# exec()の第2引数にglobals()を渡すと、実行されたコードの変数が現在の名前空間に反映される
exec_globals = globals()
exec_globals['__file__'] = dataset_path

# dataset.pyを実行
exec(open(dataset_path).read(), exec_globals)

print("\n✅ データセット準備完了！")
print("   baseline_dataset.pickle が生成されました")


In [ ]:
# モデルと損失関数、評価関数をインポート
from 畳み込み.model import NoiseDetectionAndReconstructionModel
from 畳み込み.loss import compute_noise_detection_reconstruction_loss, create_true_mask_from_intervals
from eval import evaluate_noise_detection_reconstruction_model

print("✓ モジュールのインポート完了")

# モジュールを再読み込み（ファイルを変更した場合）
# ファイルを変更した後は、このセルを再実行してモジュールを再読み込みしてください
import importlib
import sys

modules_to_reload = [
    '畳み込み.model',
    '畳み込み.loss',
    'eval'
]

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✓ {module_name}モジュールを再読み込みしました")

# 再インポート（再読み込み後）
from 畳み込み.model import NoiseDetectionAndReconstructionModel
from 畳み込み.loss import compute_noise_detection_reconstruction_loss, create_true_mask_from_intervals
from eval import evaluate_noise_detection_reconstruction_model

print("\n✅ すべてのモジュールを再読み込み完了")


In [ ]:
# データセットクラス（ノイズ検出 + 復元タスク用）
class NoiseDetectionReconstructionDataset(Dataset):
    """ノイズ検出 + 復元タスク用データセット"""
    
    def __init__(self, noisy_data, original_data, labels):
        self.noisy_data = noisy_data  # ノイズ付きデータ
        self.original_data = original_data  # ノイズ付与前のデータ
        self.labels = labels  # ノイズ区間のインデックス
    
    def __len__(self):
        return len(self.noisy_data)
    
    def __getitem__(self, idx):
        return {
            'noisy_psd': self.noisy_data[idx],
            'original_psd': self.original_data[idx],
            'noise_intervals': self.labels[idx]
        }


In [ ]:
def validate(model, dataloader, device, num_intervals=30, points_per_interval=100):
    """検証（ノイズ検出 + 復元タスク用）"""
    model.eval()
    total_loss = 0
    total_mask_loss = 0
    total_reconstruction_loss = 0
    correct = 0
    total = 0
    
    from 畳み込み.loss import create_true_mask_from_intervals
    
    with torch.no_grad():
        for batch in dataloader:
            noisy_psd = batch['noisy_psd'].to(device)
            original_psd = batch['original_psd'].to(device)
            true_noise_intervals = batch['noise_intervals'].to(device)
            
            # モデルの出力（3つの値を返す: mask, reconstructed_intervals, reconstructed_intervals_info）
            predicted_mask, reconstructed_interval, reconstructed_intervals_info = model(noisy_psd)
            
            # 真のマスクを作成
            true_mask = create_true_mask_from_intervals(true_noise_intervals, num_intervals)
            true_mask = true_mask.to(device)
            
            # 損失を計算
            loss, mask_loss, reconstruction_loss, loss_dict = \
                compute_noise_detection_reconstruction_loss(
                    predicted_mask=predicted_mask,
                    reconstructed_interval=reconstructed_interval,
                    reconstructed_intervals_info=reconstructed_intervals_info,
                    true_mask=true_mask,
                    original_psd=original_psd,
                    noisy_psd=noisy_psd,
                    true_noise_intervals=true_noise_intervals,
                    num_intervals=num_intervals,
                    mask_loss_weight=0.5,
                    reconstruction_loss_weight=1.0,
                    points_per_interval=points_per_interval
                )
            
            total_loss += loss.item()
            total_mask_loss += mask_loss.item()
            total_reconstruction_loss += reconstruction_loss.item()
            
            # マスク予測の精度
            predicted_intervals = predicted_mask.argmax(dim=1)
            total += true_noise_intervals.size(0)
            correct += (predicted_intervals == true_noise_intervals).sum().item()
    
    avg_loss = total_loss / len(dataloader)
    avg_mask_loss = total_mask_loss / len(dataloader)
    avg_reconstruction_loss = total_reconstruction_loss / len(dataloader)
    accuracy = 100 * correct / total
    
    return avg_loss, avg_mask_loss, avg_reconstruction_loss, accuracy


In [ ]:
def train_epoch(model, dataloader, optimizer, device, num_intervals=30, points_per_interval=100):
    """1エポックの学習（ノイズ検出 + 復元タスク用）"""
    model.train()
    total_loss = 0
    total_mask_loss = 0
    total_reconstruction_loss = 0
    correct = 0
    total = 0
    
    from 畳み込み.loss import create_true_mask_from_intervals
    
    for batch in dataloader:
        noisy_psd = batch['noisy_psd'].to(device)
        original_psd = batch['original_psd'].to(device)
        true_noise_intervals = batch['noise_intervals'].to(device)
        
        # 順伝播
        optimizer.zero_grad()
        predicted_mask, reconstructed_interval, reconstructed_intervals_info = model(noisy_psd)
        
        # NaN/Infチェック
        if torch.isnan(predicted_mask).any() or torch.isinf(predicted_mask).any():
            print(f"警告: マスク予測にNaN/Infが検出されました")
            continue
        
        if torch.isnan(reconstructed_interval).any() or torch.isinf(reconstructed_interval).any():
            print(f"警告: 復元出力にNaN/Infが検出されました")
            continue
        
        # 真のマスクを作成
        true_mask = create_true_mask_from_intervals(true_noise_intervals, num_intervals)
        true_mask = true_mask.to(device)
        
        # 損失を計算
        loss, mask_loss, reconstruction_loss, loss_dict = \
            compute_noise_detection_reconstruction_loss(
                predicted_mask=predicted_mask,
                reconstructed_interval=reconstructed_interval,
                reconstructed_intervals_info=reconstructed_intervals_info,
                true_mask=true_mask,
                original_psd=original_psd,
                noisy_psd=noisy_psd,
                true_noise_intervals=true_noise_intervals,
                num_intervals=num_intervals,
                mask_loss_weight=0.5,
                reconstruction_loss_weight=1.0,
                points_per_interval=points_per_interval
            )
        
        # NaN/Infチェック
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"警告: 損失にNaN/Infが検出されました")
            continue
        
        # 逆伝播
        loss.backward()
        
        # 勾配クリッピング（勾配爆発を防ぐ）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # 統計
        total_loss += loss.item()
        total_mask_loss += mask_loss.item()
        total_reconstruction_loss += reconstruction_loss.item()
        
        # マスク予測の精度
        predicted_intervals = predicted_mask.argmax(dim=1)
        total += true_noise_intervals.size(0)
        correct += (predicted_intervals == true_noise_intervals).sum().item()
    
    avg_loss = total_loss / len(dataloader)
    avg_mask_loss = total_mask_loss / len(dataloader)
    avg_reconstruction_loss = total_reconstruction_loss / len(dataloader)
    accuracy = 100 * correct / total
    
    return avg_loss, avg_mask_loss, avg_reconstruction_loss, accuracy


In [ ]:
# ハイパーパラメータ
NUM_INTERVALS = 30  # 30区間
POINTS_PER_INTERVAL = 100  # 1区間あたり100ポイント

# データセットの読み込み
print("データセットを読み込み中...")
with open('baseline_dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

train_noisy = dataset['train']['noisy_data']
train_original = dataset['train']['original_data']
train_labels = dataset['train']['labels']
val_noisy = dataset['val']['noisy_data']
val_original = dataset['val']['original_data']
val_labels = dataset['val']['labels']
test_noisy = dataset['test']['noisy_data']
test_original = dataset['test']['original_data']
test_labels = dataset['test']['labels']

print(f"訓練データ: {len(train_noisy):,}サンプル")
print(f"検証データ: {len(val_noisy):,}サンプル")
print(f"テストデータ: {len(test_noisy):,}サンプル")

# データをテンソルに変換
train_noisy = torch.FloatTensor(train_noisy)
train_original = torch.FloatTensor(train_original)
train_labels = torch.LongTensor(train_labels)
val_noisy = torch.FloatTensor(val_noisy)
val_original = torch.FloatTensor(val_original)
val_labels = torch.LongTensor(val_labels)
test_noisy = torch.FloatTensor(test_noisy)
test_original = torch.FloatTensor(test_original)
test_labels = torch.LongTensor(test_labels)

# ラベルの確認
print(f"\nラベルの確認:")
print(f"  訓練データのラベル範囲: {train_labels.min().item()} - {train_labels.max().item()}")
print(f"  期待される範囲: 0 - {NUM_INTERVALS-1}")
if train_labels.max().item() >= NUM_INTERVALS:
    raise ValueError(f"エラー: ラベルの最大値({train_labels.max().item()})がNUM_INTERVALS({NUM_INTERVALS})以上です。")
print("✓ ラベルが正しい範囲内です")

# データの前処理確認（dataset.pyで既に前処理済み）
print("\nデータの前処理確認:")
print(f"  ノイズ付きデータの平均: {train_noisy.mean():.6f}, 標準偏差: {train_noisy.std():.6f}")
print(f"  元のデータの平均: {train_original.mean():.6f}, 標準偏差: {train_original.std():.6f}")
print("✓ データは既に前処理済みです（dataset.pyでスケーリング + ログ変換 + 正規化済み）")


In [ ]:
# ハイパーパラメータ
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
NUM_INTERVALS = 30
POINTS_PER_INTERVAL = 100

# モデルの作成
model = NoiseDetectionAndReconstructionModel(
    num_intervals=NUM_INTERVALS,
    points_per_interval=POINTS_PER_INTERVAL
).to(device)
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

# DataLoaderの作成
train_dataset = NoiseDetectionReconstructionDataset(train_noisy, train_original, train_labels)
val_dataset = NoiseDetectionReconstructionDataset(val_noisy, val_original, val_labels)
test_dataset = NoiseDetectionReconstructionDataset(test_noisy, test_original, test_labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# オプティマイザとスケジューラ
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print("\n✓ モデルと学習設定の準備完了")


In [ ]:
# モジュールを再読み込み（学習前に最新のコードを確実に読み込む）
# ファイルを変更した後は、このセルを再実行してモジュールを再読み込みしてください
import importlib
import sys

modules_to_reload = [
    '畳み込み.model',
    '畳み込み.loss',
    'eval'
]

print("=" * 60)
print("モジュール再読み込み中...")
print("=" * 60)

for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✓ {module_name}モジュールを再読み込みしました")
    else:
        print(f"⚠ {module_name}モジュールはまだ読み込まれていません")

# 再インポート（再読み込み後）
from 畳み込み.model import NoiseDetectionAndReconstructionModel
from 畳み込み.loss import compute_noise_detection_reconstruction_loss, create_true_mask_from_intervals
from eval import evaluate_noise_detection_reconstruction_model

print("\n✅ すべてのモジュールを再読み込み完了")
print("=" * 60)


In [ ]:
# 学習実行
# 学習履歴
train_losses = []
train_mask_losses = []
train_reconstruction_losses = []
train_accuracies = []
val_losses = []
val_mask_losses = []
val_reconstruction_losses = []
val_accuracies = []
best_val_acc = 0
best_model_state = None
patience = 10  # Early Stoppingのpatience
patience_counter = 0

print("学習開始...")
print("=" * 60)
print(f"総エポック数: {NUM_EPOCHS}")

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # 学習
    train_loss, train_mask_loss, train_reconstruction_loss, train_acc = train_epoch(
        model, train_loader, optimizer, device, NUM_INTERVALS, POINTS_PER_INTERVAL
    )
    
    # 検証
    val_loss, val_mask_loss, val_reconstruction_loss, val_acc = validate(
        model, val_loader, device, NUM_INTERVALS, POINTS_PER_INTERVAL
    )
    
    # 学習率の調整
    scheduler.step(val_loss)
    
    # 履歴の保存
    train_losses.append(train_loss)
    train_mask_losses.append(train_mask_loss)
    train_reconstruction_losses.append(train_reconstruction_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_mask_losses.append(val_mask_loss)
    val_reconstruction_losses.append(val_reconstruction_loss)
    val_accuracies.append(val_acc)
    
    # ベストモデルの保存とEarly Stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    # Early Stoppingチェック
    if patience_counter >= patience:
        print(f"\n⚠ Early Stopping: {patience}エポック連続で改善なし。学習を停止します。")
        print(f"   ベスト検証精度: {best_val_acc:.2f}% (エポック {epoch+1-patience})")
        break
    
    # ログ出力
    if (epoch + 1) % 1 == 0 or epoch == 0 or (epoch + 1) == NUM_EPOCHS:
        epoch_time = time.time() - epoch_start
        elapsed_time = time.time() - start_time
        remaining_epochs = NUM_EPOCHS - (epoch + 1)
        estimated_remaining = (elapsed_time / (epoch + 1)) * remaining_epochs if epoch > 0 else 0
        
        current_lr = optimizer.param_groups[0]["lr"]
        
        # モデルの出力を確認（最初のエポックのみ）
        if epoch == 0:
            model.eval()
            with torch.no_grad():
                sample_noisy = train_noisy[:1].to(device)
                sample_mask, sample_reconstructed, sample_reconstructed_info = model(sample_noisy)
                print(f"\n=== エポック {epoch+1} の詳細 ===")
                print(f"  学習率: {current_lr:.6f}")
                print(f"  サンプルマスク予測: {sample_mask[0].argmax().item()} (確信度: {sample_mask[0].max().item():.4f})")
                print(f"  復元区間の形状: {sample_reconstructed.shape}")
                print(f"  復元区間情報: start={sample_reconstructed_info[0, 0].item()}, end={sample_reconstructed_info[0, 1].item()}, predicted={sample_reconstructed_info[0, 2].item()}")
            model.train()
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] ({epoch_time:.1f}秒)")
        print(f"  Train Loss: {train_loss:.4f} (Mask: {train_mask_loss:.4f}, Recon: {train_reconstruction_loss:.4f}), Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f} (Mask: {val_mask_loss:.4f}, Recon: {val_reconstruction_loss:.4f}), Val Acc: {val_acc:.2f}%")
        print(f"  学習率: {current_lr:.6f}")
        if epoch > 0:
            val_loss_change = val_losses[-1] - val_losses[-2] if len(val_losses) > 1 else 0
            val_acc_change = val_accuracies[-1] - val_accuracies[-2] if len(val_accuracies) > 1 else 0
            print(f"  検証損失の変化: {val_loss_change:+.4f}, 検証精度の変化: {val_acc_change:+.2f}%")
        if estimated_remaining > 0:
            print(f"  経過時間: {elapsed_time/60:.1f}分, 残り見積もり: {estimated_remaining/60:.1f}分")
        print()

total_time = time.time() - start_time
print(f"学習完了！総時間: {total_time/60:.1f}分 ({total_time/3600:.2f}時間)")


In [ ]:
# ============================================================
# 評価用セル（独立して実行可能）
# 変数が消えても、このセルだけ実行すれば評価できます
# ============================================================

import os
import sys
import importlib
import torch
import pickle
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# パス設定（必要に応じて変更）
project_root = '/content/drive/MyDrive/soturon'
pickle_path = f'{project_root}/baseline_dataset.pickle'
model_path = f'{project_root}/畳み込み/noise_detection_reconstruction_model.pth'

# モジュールを再読み込み（最新のコードを確実に読み込む）
modules_to_reload = ['畳み込み.model', '畳み込み.loss', 'eval']
for module_name in modules_to_reload:
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f"✓ {module_name}モジュールを再読み込みしました")

# インポート
from 畳み込み.model import NoiseDetectionAndReconstructionModel
from eval import evaluate_noise_detection_reconstruction_model

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ハイパーパラメータ設定
NUM_INTERVALS = 30
POINTS_PER_INTERVAL = 100
BATCH_SIZE = 64

# データセットの読み込み
print("データセットを読み込み中...")
with open(pickle_path, 'rb') as f:
    dataset = pickle.load(f)

test_noisy = torch.FloatTensor(dataset['test']['noisy_data'])
test_original = torch.FloatTensor(dataset['test']['original_data'])
test_labels = torch.LongTensor(dataset['test']['labels'])

# データセットクラス
class NoiseDetectionReconstructionDataset:
    def __init__(self, noisy_data, original_data, labels):
        self.noisy_data = noisy_data
        self.original_data = original_data
        self.labels = labels
    
    def __len__(self):
        return len(self.noisy_data)
    
    def __getitem__(self, idx):
        return {
            'noisy_psd': self.noisy_data[idx],
            'original_psd': self.original_data[idx],
            'noise_intervals': self.labels[idx]
        }

test_dataset = NoiseDetectionReconstructionDataset(test_noisy, test_original, test_labels)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# モデルの初期化と重みの読み込み
model = NoiseDetectionAndReconstructionModel(
    num_intervals=NUM_INTERVALS,
    points_per_interval=POINTS_PER_INTERVAL
).to(device)

if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✓ モデルの重みを読み込みました: {model_path}")
    if 'val_accuracy' in checkpoint:
        print(f"  保存時の検証精度: {checkpoint['val_accuracy']:.2f}%")
else:
    print(f"⚠️ モデルの重みファイルが見つかりません: {model_path}")
    print("学習を実行してモデルを保存してください。")

print("\n✅ 評価の準備が完了しました")
print("=" * 60)

# 評価を実行
print("\nテストデータで評価中...")
results = evaluate_noise_detection_reconstruction_model(
    model, test_loader, device=device, num_intervals=NUM_INTERVALS, points_per_interval=POINTS_PER_INTERVAL
)

print(f"\n=== 評価結果 ===")
print(f"  マスク予測精度: {results['mask_accuracy']:.4f}")
print(f"  マスク予測 Precision: {results['mask_precision']:.4f}")
print(f"  マスク予測 Recall: {results['mask_recall']:.4f}")
print(f"  マスク予測 F1-score: {results['mask_f1_score']:.4f}")
if results['loss'] != float('inf'):
    print(f"  復元損失（MSE損失、lossキー）: {results['loss']:.6f}")
    if results['reconstruction_accuracy'] is not None:
        print(f"  復元精度: {results['reconstruction_accuracy']:.2f}%")
else:
    print(f"  復元損失: 計算できませんでした（マスク予測が正しいサンプルがありませんでした）")
print(f"  マスク予測が正しいサンプル数: {results['num_correct_masks']} / {len(results['labels'])}")
print(f"\n  ※ 復元損失（lossキー）: {results['loss']:.6f} - 最も重要な指標")
if results['reconstruction_accuracy'] is not None:
    print(f"  ※ 復元精度: {results['reconstruction_accuracy']:.2f}% - 復元できた割合")


In [ ]:
# 学習曲線を可視化（復元損失のみ）
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# 復元損失
ax.plot(train_reconstruction_losses, label='Train Recon Loss', color='blue', linewidth=2)
ax.plot(val_reconstruction_losses, label='Val Recon Loss', color='red', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Reconstruction Loss', fontsize=12)
ax.set_title('Reconstruction Loss', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
print("学習曲線を 'training_curves.png' に保存しました")

plt.show()


In [ ]:
# ベストモデルを保存
if best_model_state is not None:
    torch.save({
        'model_state_dict': best_model_state,
        'val_accuracy': best_val_acc,
        'num_intervals': NUM_INTERVALS,
        'points_per_interval': POINTS_PER_INTERVAL,
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }, 'noise_detection_reconstruction_model.pth')
    print("ベストモデルを 'noise_detection_reconstruction_model.pth' に保存しました")
else:
    print("警告: ベストモデルの状態が保存されていません")


## 📥 結果ファイルのダウンロード

以下のセルを実行して、結果ファイルをダウンロードできます：
- `noise_detection_reconstruction_model.pth` (学習済みモデル)
- `training_curves.png` (学習曲線)


In [ ]:
# 結果ファイルのダウンロード
from google.colab import files

files.download('noise_detection_reconstruction_model.pth')
files.download('training_curves.png')
print("✓ ダウンロード完了！")
